### 11.1.2. 수렴하지 않는 활성화 함수 : Gradient 폭주/소실 방지 
- kernel_initializer = he_normal
- kernel_initializer = lecun_normal 

- 활성화 함수를 잘못 선택하면 자칫 그레이디언트의 소실이나 폭주
- 은닉층 활성화 함수 성능 
    - SELU > ELU > LeakyReLU(그리고 변종들) > ReLU > tanh> 로지스틱 순
    
- 과대적합 -> RReLU ( Randomized ReLU ) 
- 큰 훈련세트 -> PReLU 

- LeakyReLu(z) = $ max(\alpha, z) $ 
- RReLU : $\alpha$가 훈련하는 동안 학습
- PReLu : $\alpha$ 무작위 선택, 테스트시 $\alpha$ 평균 사용

 
- SELU : 네트워크 자기 정규화, 출력 ~ N(0, 1) 
- ELU: ? 

In [1]:
import tensorflow as tf 
from tensorflow import keras 

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 

In [2]:
housing = fetch_california_housing() 

In [3]:
housing.data.shape

(20640, 8)

In [4]:
# 테스트, 사용 세트 분리 
X_train_full, X_test, y_train_full, y_test =\
    train_test_split(housing.data, housing.target)

# 학습, 검증 셋 분리 
X_train, X_valid, y_train, y_valid =\
    train_test_split(X_train_full, y_train_full)

In [5]:
X_train.shape

(11610, 8)

In [6]:
y_train.shape

(11610,)

In [7]:
y_train_full.shape

(15480,)

In [8]:
X_test.shape

(5160, 8)

In [9]:
y_test.shape

(5160,)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

###### model 생성 
- 글로럿/세이비어/르쿤 초기화 
    - $ fan_{avg} = {{fan_{in} +fan_{out}}) \over 2 }$
- PReLU 
- selu 
- SELU > ELU > LeakyReLU(그리고 변종들) > ReLU > tanh> 로지스틱 순

In [11]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation = 'relu',
                      input_shape = X_train.shape[1:]),
    
    # LeakyRelu 
    # 층 생성 후, 적용할 층 뒤에 LeakyReLU 적용  
    keras.layers.Dense(10, kernel_initializer = 'he_normal'),
    
    keras.layers.LeakyReLU(alpha = 0.3),
    
    # SELU 
    keras.layers.Dense(10, kernel_initializer = 'he_normal'),
    
    # PReLU 
    keras.layers.PReLU(alpha_initializer='zeros'),
    
    # SELU 
    keras.layers.Dense(10, activation = 'selu', 
                      kernel_initializer = 'lecun_normal')
])

### 11.1.3. 배치 정규화 

- 훈련초기, 그레디언트 소실, 폭주 감소시킬 수 있음
    - kernel_initializer = he_normal 
    - kernel_initializer = lecun_normal 
    - 하지만 훈련하는 동안, 다시 폭주/소실 발생 가능 
- 훈련 중, 그레디언트 소실, 폭주 방지 
    - 배치 정규화(BN) 
        - 각 층에서 활성화 함수를 통과하기 전, 후 모델에 연산 추가 

- γ(출력 스케일 벡터)와 β(출력 이동 벡터)는 일반적인 역전파를 통해 학습
- μ(최종 입력 평균 벡터)와 σ(최종 입력 표준편 차 벡터)는 지수 이동 평균을 사용
    - 각 층마다 4개의 파라미터 γ, β, μ, σ 추가

Training 할 때는 mini-batch의 평균과 분산으로 normalize 하고,  
Test 할 때는 계산해놓은 이동 평균으로 normalize 한다.

- 역전파 학습 :   
    γ(출력 스케일 벡터) / β(출력 이동 벡터) ➤ 역전파 통해 추정 

- 지수 이동 평균을 통해 추정  
    μ(최종 입력 평균 벡터) / σ(최종 입력 표준편 차 벡터) ➤ 지수 이동 평균 사용 추정 

- 중요 파라미터 
    - axis : 학습시키는 데이터의 컬럼 / 피쳐 

In [12]:
import tensorflow as tf 
from tensorflow import keras 

In [13]:
### 데이터 적재 
# fashion dataset : 60000 x 28 x 28 크기 
fashion_mnist = keras.datasets.fashion_mnist 
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [14]:
### 훈련세트와 테스트 세트 분리 
X_valid = X_train_full[:5000] / 255.0

X_train = X_train_full[5000:] / 255.0

y_valid = y_train_full[:5000] / 255.0

y_train = y_train_full[5000:] / 255.0

X_test = X_test / 255.0 

In [15]:
X_train.shape

(55000, 28, 28)

In [16]:
y_train.shape

(55000,)

###### 활성화 함수 이전, 배치 정규과화

In [17]:
# 배치정규화 추가 
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),    
    
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer = 'he_normal'),
    
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation='elu', kernel_initializer = 'he_normal'),

    keras.layers.BatchNormalization(), 
    keras.layers.Dense(10, activation = 'softmax')
])

###### 활성화 함수 이후, 배치 정규화 : keras.layers.BatchNormalization() 

In [18]:
X_train.shape

(55000, 28, 28)

In [19]:
y_train.shape

(55000,)

In [20]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[28, 28]), 
    
#     keras.layers.BatchNormalization(),
    
    # 은닉층 : 활성화 함수 지정 안함, use_bias = False
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False), 
    # 배치 정규화 층 입력
    keras.layers.BatchNormalization(),
    # 활성함수 입력 
    keras.layers.Activation("elu"),
    
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False), 
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    
    keras.layers.Dense(10, activation="softmax")
])


In [21]:
model.summary()

# 정규화 층 : 784 * 4개의 파라미터가 추가됨 

'''    
- 아래 그림 
    - 정규화 배치 층 파라미터 : 784 * 4 ( γ, β, μ, σ ) = 3136 
    - gamma : γ / beta : β, 배치 정규화 층에서 mo
'''

 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 300)               235200    
_________________________________________________________________
batch_normalization_3 (Batch (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               30000     
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
_________________________________________________________________
activation_1 (Activation)    (None, 100)              

'    \n- 아래 그림 \n    - 정규화 배치 층 파라미터 : 784 * 4 ( γ, β, μ, σ ) = 3136 \n    - gamma : γ / beta : β, 배치 정규화 층에서 mo\n'

### 그레이디언트 클리핑 : 배치 정규화가 어려울 때, 그레디언트 < threshold 되도록 clipping 하는 것 

In [22]:
# Clipping 
optimizer = keras.optimizers.SGD(clipvalue=1.0) 

# Clip norm 
# l2 norm이 지정한 임곗값보다 클 경우, 그레디언트 클리핑 
# optimizer = keras.optimizers.SGD(clipnorm=1.0)

model.compile(loss ='sparse_categorical_crossentropy',    
              optimizer=optimizer,
              metrics = ['accuracy']
             )

In [23]:
model.fit(X_train, y_train, 
         epochs = 10)

Train on 55000 samples
Epoch 1/10
55000/55000 [==============================] - 5s 98us/sample - loss: 0.0787 - accuracy: 0.0998
Epoch 2/10
55000/55000 [==============================] - 5s 92us/sample - loss: 0.0034 - accuracy: 0.1008
Epoch 3/10
55000/55000 [==============================] - 5s 91us/sample - loss: 0.0018 - accuracy: 0.1008
Epoch 4/10
55000/55000 [==============================] - 5s 83us/sample - loss: 0.0012 - accuracy: 0.1008
Epoch 5/10
55000/55000 [==============================] - 5s 82us/sample - loss: 9.1240e-04 - accuracy: 0.1008
Epoch 6/10
55000/55000 [==============================] - 4s 76us/sample - loss: 7.2548e-04 - accuracy: 0.1008
Epoch 7/10
55000/55000 [==============================] - 4s 77us/sample - loss: 5.9903e-04 - accuracy: 0.1008
Epoch 8/10
55000/55000 [==============================] - 4s 76us/sample - loss: 5.0931e-04 - accuracy: 0.1008
Epoch 9/10
55000/55000 [==============================] - 5s 83us/sample - loss: 4.4251e-04 - accuracy: 0

## 11.2 사전훈련된 층 재사용하기 : 전이학습  ( MNIST Fashion 데이터셋 재사용 ) 
- 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아본 다음(14장 참조), 그 신경망의 하위층을 재사용

###### 사전 모델 훈련

In [24]:
import numpy as np

def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [25]:
X_train_A.shape

(55000, 28, 28)

In [26]:
X_train_B.shape

(0, 28, 28)

In [27]:
tf.random.set_seed(42)
np.random.seed(42)

In [28]:
# model building e
model_A = keras.models.Sequential()

model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
    
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [29]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [30]:
history = model_A.fit(X_train_A, y_train_A, epochs=5,
                    validation_data=(X_valid_A, y_valid_A))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 6s 115us/sample - loss: 0.0116 - accuracy: 0.1007 - val_loss: 0.0015 - val_accuracy: 0.0914
Epoch 2/5
55000/55000 [==============================] - 5s 95us/sample - loss: 0.0011 - accuracy: 0.1008 - val_loss: 7.3474e-04 - val_accuracy: 0.0914
Epoch 3/5
55000/55000 [==============================] - 5s 92us/sample - loss: 6.4517e-04 - accuracy: 0.1008 - val_loss: 4.8310e-04 - val_accuracy: 0.0914
Epoch 4/5
55000/55000 [==============================] - 5s 91us/sample - loss: 4.5314e-04 - accuracy: 0.1008 - val_loss: 3.5847e-04 - val_accuracy: 0.0914
Epoch 5/5
55000/55000 [==============================] - 5s 97us/sample - loss: 3.4859e-04 - accuracy: 0.1008 - val_loss: 2.8431e-04 - val_accuracy: 0.0914


In [31]:
model_A.save("my_model_A.h5")

In [32]:
model_B = keras.models.Sequential()

model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
    
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [33]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=5,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/5


In [ ]:
model.summary()

###### 재사용 
- 모델B.clone_model(모델A) 
- 모델B.set_weights(모델A.get_weights) 

In [ ]:
# 모델 읽어옴 
model_A = keras.models.load_model("my_model_A.h5")

# model_B_on_A와 model_A는 층을 공유
# 작업 B를 위한 모델 : model_B_on_A 
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

# 출력층 레이어 추가 
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
# model_B_on_A와 model_A는 층을 공유
model_A_clone = keras.models.clone_model(model_A)

# model_A의 가중치를 가져옴 
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
# model_B_on_A 새로운 레이어 : 입력 레이어 ~ 출력 직전 레이어 학습 동결 
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

# 출력 직전 레이어 ~ 출력 레이어( 새로운 레이어 ) 가중치 학습 
# 에포크 약간만 주고 학습 
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-2),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

In [ ]:
# 재사용된 층의 동결 해제, 모델 다시 학습 - 재사용된 층을 현재 학습을 튜닝하기 위해 훈련 
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

# 재사용된 층의 동결을 해제 후, 학습률을 낮추어 재사용된 가중치 유지 
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-4),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)